In [ ]:
import nbformat

notebook_path = "Fusion Retrieval.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")


In [1]:
!pip install langchain-community langchain-text-splitters langchain-ollama pypdf pymupdf langchain-huggingface sentence-transformers langchain-experimental faiss-cpu rank-bm25  --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 81.4 MB/s eta 0:00:00


In [2]:
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from typing import List
import numpy as np
from rank_bm25 import BM25Okapi
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [3]:
embedding_model=HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={"device":"cuda"},
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [39]:
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
chunk_size=1024
chunk_overlap=64
query="What the CRAG proposed to improve "

In [40]:
def replace_tab_into_space(list_of_docs):
    for doc in list_of_docs:
        doc.page_content=doc.page_content.replace("\t"," ")
    return list_of_docs

def encode_pdf_and_get_splits_documents(path,chunk_size,chunk_overlap):

    loader=PyPDFLoader(path)
    documents=loader.load()

    splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunks=splitter.split_documents(documents)

    clean_chunks=replace_tab_into_space(chunks)

    vectorstore=FAISS.from_documents(clean_chunks,embedding_model)

    return vectorstore,clean_chunks


def create_bm25_index(documents:List[Document]) ->BM25Okapi:
    tokenized_docs=[doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

def show_context(context):
    for i,c in enumerate(context):
        print(f"context{i+1} ")
        print(c)
        print("\n")


In [41]:
vectorstore,clean_chunks=encode_pdf_and_get_splits_documents(
    path,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
bm25=create_bm25_index(clean_chunks)



In [42]:
def fusion_retrieval(vectorstore, bm25, query, k=3, alpha=0.5):
    epsilon = 1e-8
    retriever=vectorstore.as_retriever()
    all_docs = retriever.invoke("", k=vectorstore.index.ntotal)
    bm25_scores=bm25.get_scores(query.split()) #query的tokenizer要和前面的bm25一樣

    vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs))
    #  Normalize scores
    vector_scores = np.array([score for _, score in vector_results])

    #因為距離越短越好所以用1-X_normalization，分數越高越好所以不用

    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) -  np.min(bm25_scores) + epsilon)

    #  Combine scores
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores

    #  Rank documents
    sorted_indices = np.argsort(combined_scores)[::-1]

    #  Return top k documents
    return [all_docs[i] for i in sorted_indices[:k]]





In [45]:
# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=3, alpha=0.1)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)

context1 
retrieval evaluator is designed to assess the
overall quality of retrieved documents for a
query, returning a confidence degree based
on which different knowledge retrieval ac-
tions can be triggered. Since retrieval from
static and limited corpora can only return sub-
optimal documents, large-scale web searches
are utilized as an extension for augmenting the
retrieval results. Besides, a decompose-then-
recompose algorithm is designed for retrieved
documents to selectively focus on key infor-
mation and filter out irrelevant information in
them. CRAG is plug-and-play and can be
seamlessly coupled with various RAG-based
approaches. Experiments on four datasets
covering short- and long-form generation tasks
show that CRAG can significantly improve the
performance of RAG-based approaches. 1
1 Introduction
Large language models (LLMs) have attracted
increasing attention and exhibited impressive abili-
ties to understand instructions and generate fluent
language texts (Brown et a